In [1]:
## Path for modules

import sys

import numpy as np
import xarray as xr

from netCDF4 import Dataset

sys.path.insert(0,"/mnt/meom/workdir/henelle/Notebooks/git/xscale")
import xscale
import xscale.spectral.fft as xfft

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import numpy.ma as ma

import matplotlib.cm as mplcm
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic

import matplotlib.pyplot as plt
import matplotlib as mpl
from datetime import date, datetime

import pandas as pd

import seaborn as sns
sns.set(color_codes=True)

%matplotlib inline

In [2]:
## Dataset

dirfilt_00 = "/mnt/meom/workdir/henelle/eNATL60/eNATL60-BLB002-S/1h/ACO/"
dirfilt_T0 = "/mnt/meom/workdir/henelle/eNATL60/eNATL60-BLBT02-S/1h/ACO/"

In [3]:
## All files, both seasons

#tfile_strain00 = dirfilt_00 + 'eNATL60ACO-BLB002_*.1h_strain10m.nc'
#tfile_strainT0 = dirfilt_T0 + 'eNATL60ACO-BLBT02_*.1h_strain10m.nc'
#tfile_curl00   = dirfilt_00 + 'eNATL60ACO-BLB002_*.1h_curl10m.nc'
#tfile_curlT0   = dirfilt_T0 + 'eNATL60ACO-BLBT02_*.1h_curl10m.nc'

In [4]:
## All files, JAS

tfile_JAS_strain00 = dirfilt_00 + 'eNATL60ACO-BLB002_y2009*.1h_strain10m.nc'
tfile_JAS_strainT0 = dirfilt_T0 + 'eNATL60ACO-BLBT02_y2009*.1h_strain10m.nc'
tfile_JAS_curl00   = dirfilt_00 + 'eNATL60ACO-BLB002_y2009*.1h_curl10m.nc'
tfile_JAS_curlT0   = dirfilt_T0 + 'eNATL60ACO-BLBT02_y2009*.1h_curl10m.nc'

In [5]:
## All files, JFM

#tfile_JFM_strain00 = dirfilt_00 + 'eNATL60ACO-BLB002_y2010*.1h_strain10m.nc'
#tfile_JFM_strainT0 = dirfilt_T0 + 'eNATL60ACO-BLBT02_y2010*.1h_strain10m.nc'
#tfile_JFM_curl00   = dirfilt_00 + 'eNATL60ACO-BLB002_y2010*.1h_curl10m.nc'
#tfile_JFM_curlT0   = dirfilt_T0 + 'eNATL60ACO-BLBT02_y2010*.1h_curl10m.nc'

In [6]:
with xr.open_dataset(dirfilt_00 + 'eNATL60ACO-BLB002_y2009m07d01.1h_strain10m.nc') as strain_JAS_00:
        
    navlat   = strain_JAS_00['nav_lat'].values
    navlon   = strain_JAS_00['nav_lon'].values
    #time_JAS = strain_JAS_00['time_counter'].values

In [7]:
#with xr.open_dataset(dirfilt_00+'eNATL60ACO-BLB002_y2010m01d01.1h_strain10m.nc') as strain_JFM_00:

#    time_JFM = strain_JFM_00['time_counter'].values

In [8]:
#navlon = xr.DataArray(navlon)
#navlat = xr.DataArray(navlat)

In [9]:
## Selecting box

box    = (-31.5,-27,30.6,36)
domain = (box[0] < navlon)*(navlon < box[1])*(box[2] < navlat)*(navlat < box[3])
where  = np.where(domain)

lats = navlat[where]
lons = navlon[where]


ind  = np.unravel_index(np.argmin(lats, axis=None), lats.shape)
jmin = where[0][ind[0]]
ind  = np.unravel_index(np.argmax(lats, axis=None), lats.shape)
jmax = where[0][ind[0]]
ind  = np.unravel_index(np.argmin(lons, axis=None), lons.shape)
imin = where[1][ind[0]]
ind  = np.unravel_index(np.argmax(lons, axis=None), lons.shape)
imax = where[1][ind[0]]

In [10]:
print('jmin = '+str(jmin)+'; jmax = '+str(jmax))
print('imin = '+str(imin)+'; imax = '+str(imax))

jmin = 499; jmax = 882
imin = 354; imax = 623


In [11]:
def fct_open_datasets(path,var,jmin,jmax,imin,imax):
    
    with xr.open_mfdataset(path,combine='by_coords') as file:
        
        varvalues = file[var][:,jmin:jmax+1,imin:imax+1]
        time      = file['time_counter']
        
    return varvalues,time

strain_00,_,_,_            = fct_open_dataset(tfile_strain00,'sostrainoverf')
strain_T0,_,_,_            = fct_open_dataset(tfile_strainT0,'sostrainoverf')
curl_00,_,_,_              = fct_open_dataset(tfile_curl00,  'socurloverf')
curl_T0,navlat,navlon,time = fct_open_dataset(tfile_curlT0,  'socurloverf')

In [12]:
strain_JAS_00 = fct_open_datasets(tfile_JAS_strain00,'sostrainoverf',jmin,jmax,imin,imax)
strain_JAS_T0 = fct_open_datasets(tfile_JAS_strainT0,'sostrainoverf',jmin,jmax,imin,imax)
curl_JAS_00   = fct_open_datasets(tfile_JAS_curl00,  'socurloverf',jmin,jmax,imin,imax)
curl_JAS_T0   = fct_open_datasets(tfile_JAS_curlT0,  'socurloverf',jmin,jmax,imin,imax)

In [19]:
#strain_JAS_00.pop()


AttributeError: 'tuple' object has no attribute 'pop'

In [14]:
#strain_JAS_00 = xr.DataArray(strain_JAS_00)
#strain_JAS_T0 = xr.DataArray(strain_JAS_T0)
#curl_JAS_00   = xr.DataArray(curl_JAS_00)
#curl_JAS_T0   = xr.DataArray(curl_JAS_T0)

strain_JFM_00,_,_,_            = fct_open_dataset(tfile_JFM_strain00,'sostrainoverf')
strain_JFM_T0,_,_,_            = fct_open_dataset(tfile_JFM_strainT0,'sostrainoverf')
curl_JFM_00,_,_,_              = fct_open_dataset(tfile_JFM_curl00,  'socurloverf')
curl_JFM_T0,_,_,_              = fct_open_dataset(tfile_JFM_curlT0,  'socurloverf')

In [15]:
## Selecting data in the box

curl_filt_box_T0   = np.mean(curl_JAS_T0.stack(z=('x', 'y')),axis=0)
curl_filt_box_00   = np.mean(curl_JAS_00.stack(z=('x', 'y')),axis=0)
strain_filt_box_T0 = np.mean(strain_JAS_T0.stack(z=('x', 'y')),axis=0)
strain_filt_box_00 = np.mean(strain_JAS_00.stack(z=('x', 'y')),axis=0)

AttributeError: 'tuple' object has no attribute 'stack'

In [ ]:
def hist(varT0,var00,var_name,season,lonmin,lonmax,latmin,latmax):
    
    fig = plt.figure(figsize=(14,8))
    axes1 = fig.add_subplot(1,1,1)

    axes1.hist(varT0,100,alpha=0.5,range=(-1,1),color='r',label='tide')
    axes1.hist(var00,100,alpha=0.5,range=(-1,1),color='b',label='no tide')

    plt.xlim(-0.5,0.5)
    plt.tick_params(labelsize=14)
    plt.ticklabel_format(style='sci',axis='x',scilimits=(0,0))
    plt.xlabel('24h filtered '+var_name+' @ 10 m',size=14)
    plt.title(season,size=16)
    plt.text(-0.48,7000,'Region:\n \n '+str(lonmin)+'$^o$W to '+str(lonmax)+'$^o$W \n '
             +str(latmin)+'$^o$N to '+str(latmax)+''$^o$N',
             size=16,bbox=dict(boxstyle="square",
                       ec=(0.6, 0.8, 0.8),
                       fc=(0.6, 0.8, 0.8),
                       ))
    plt.legend(fontsize=14,loc=2,shadow=True,edgecolor='dimgray')

    #plt.savefig('./PDF/curl_JAS_(-31,-28,33,36).png')
    

In [ ]:
hist(curl_filt_box_T0,curl_filt_box_T0,'curl/f','July - August - September',-31.5,-27,30.6,36)

## Histogram of PDF - curl

fig = plt.figure(figsize=(14,8))
axes1 = fig.add_subplot(1,1,1)

axes1.hist(curl_filt_box_T0,100,alpha=0.5,range=(-1,1),color='r',label='tide')
axes1.hist(curl_filt_box_00,100,alpha=0.5,range=(-1,1),color='b',label='no tide')

plt.xlim(-0.5,0.5)
plt.tick_params(labelsize=14)
plt.ticklabel_format(style='sci',axis='x',scilimits=(0,0))
plt.xlabel('24h filtered curl/f @ 10 m',size=14)
plt.title('July - August - September',size=16)
plt.text(-0.48,7000,'Region:\n \n 31$^o$W to 28$^o$W \n 33$^o$N to 36$^o$N',
         size=16,bbox=dict(boxstyle="square",
                   ec=(0.6, 0.8, 0.8),
                   fc=(0.6, 0.8, 0.8),
                   ))
plt.legend(fontsize=14,loc=2,shadow=True,edgecolor='dimgray')

#plt.savefig('./PDF/curl_JAS_(-31,-28,33,36).png')

## Histogram of PDF - strain

fig = plt.figure(figsize=(14,8))
axes1 = fig.add_subplot(1,1,1)

axes1.hist(strain_filt_box_T0,100,alpha=0.5,range=(0,1),color='r',label='tide')
axes1.hist(strain_filt_box_00,100,alpha=0.5,range=(0,1),color='b',label='no tide')

plt.tick_params(labelsize=14)
plt.ticklabel_format(style='sci',axis='x',scilimits=(0,0))
plt.xlabel('24h filtered strain/f @ 10 m',size=14)
plt.title('July - August - September',size=16)
plt.text(0.865,8000,'Region:\n \n 31$^o$W to 28$^o$W \n 33$^o$N to 36$^o$N',
         size=16,bbox=dict(boxstyle="square",
                   ec=(0.6, 0.8, 0.8),
                   fc=(0.6, 0.8, 0.8),
                   ))

plt.legend(fontsize=14,shadow=True,edgecolor='dimgray')

#plt.savefig('./PDF/strain_JAS_(-31,-28,33,36).png')